- PassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.
- HomePlanet - The planet the passenger departed from, typically their planet of permanent residence.
- CryoSleep - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.
- Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.
- Destination - The planet the passenger will be debarking to.
- Age - The age of the passenger.
- VIP - Whether the passenger has paid for special VIP service during the voyage.
- RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.
- Name - The first and last names of the passenger.
- Transported - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.

Plan:
1) Cabin split into deck & num  & side
2) delete `Name`
3) PassengerId split into group & pp
4) CryoSleep, VIP, HomePlanet, Destination and parts of passenger and cabin to labels
5) NaNs

In [191]:
# import libraries
import numpy as np
import pandas as pd
import datetime

from sklearn.compose import make_column_transformer, make_column_selector, ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score

RANDOM_STATE = 654321

In [158]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
df = pd.concat([train, test]).reset_index()
train_index = ~df.Transported.isna()
test_index = df.Transported.isna()
print(df.info())
# print(test_index)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         12970 non-null  int64  
 1   PassengerId   12970 non-null  object 
 2   HomePlanet    12682 non-null  object 
 3   CryoSleep     12660 non-null  object 
 4   Cabin         12671 non-null  object 
 5   Destination   12696 non-null  object 
 6   Age           12700 non-null  float64
 7   VIP           12674 non-null  object 
 8   RoomService   12707 non-null  float64
 9   FoodCourt     12681 non-null  float64
 10  ShoppingMall  12664 non-null  float64
 11  Spa           12686 non-null  float64
 12  VRDeck        12702 non-null  float64
 13  Name          12676 non-null  object 
 14  Transported   8693 non-null   object 
dtypes: float64(6), int64(1), object(8)
memory usage: 1.5+ MB
None


In [159]:
def cabin2dns(row):
    # print(row['Cabin'])
    if row['Cabin'] is not np.NAN:
        a = row['Cabin'].split('/')
        row['Deck'] = a[0]
        row['Num'] = int(a[1])
        row['Side'] = a[2]
    else:
        row['Deck'] = np.NAN
        row['Num'] = np.NAN
        row['Side'] = np.NAN
    return row

In [160]:
def passenger2grouppp(row):
    if row['PassengerId'] is not np.NAN:
        a = row['PassengerId'].split('_')
        row['Group'] = int(a[0])
        row['Pp'] = int(a[1])
    else:
        row['Group'] = np.NAN
        row['Pp'] = np.NAN
    return row

In [161]:
df = df.apply(cabin2dns, axis=1)
df.drop(columns=['Cabin'], inplace=True)

In [162]:
df = df.apply(passenger2grouppp, axis=1)
# df.drop(columns=['PassengerId'], inplace=True)

In [163]:
df.drop(columns=['Name'], inplace=True)

In [164]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         12970 non-null  int64  
 1   PassengerId   12970 non-null  object 
 2   HomePlanet    12682 non-null  object 
 3   CryoSleep     12660 non-null  object 
 4   Destination   12696 non-null  object 
 5   Age           12700 non-null  float64
 6   VIP           12674 non-null  object 
 7   RoomService   12707 non-null  float64
 8   FoodCourt     12681 non-null  float64
 9   ShoppingMall  12664 non-null  float64
 10  Spa           12686 non-null  float64
 11  VRDeck        12702 non-null  float64
 12  Transported   8693 non-null   object 
 13  Deck          12671 non-null  object 
 14  Num           12671 non-null  float64
 15  Side          12671 non-null  object 
 16  Group         12970 non-null  int64  
 17  Pp            12970 non-null  int64  
dtypes: float64(7), int64(3), o

In [165]:
group_np = df[['Group', 'Pp']].groupby('Group', as_index=False).count()

# group_np

In [166]:
def count_group(row):
    global group_np
    row['Group_count'] = group_np[group_np.Group == row.Group].reset_index().Pp[0]
    return row

In [167]:
df = df.apply(count_group, axis=1)
df

/home/nikolay/.local/lib/python3.10/site-packages/IPython/lib/pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
/home/nikolay/.local/lib/python3.10/site-packages/IPython/core/formatters.py:342: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,index,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Num,Side,Group,Pp,Group_count
0,0,0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,B,0.0,P,1,1,1
1,1,0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,F,0.0,S,2,1,1
2,2,0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,A,0.0,S,3,1,2
3,3,0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,A,0.0,S,3,2,2
4,4,0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,F,1.0,S,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12965,4272,9266_02,Earth,True,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,NaN,G,1496.0,S,9266,2,2
12966,4273,9269_01,Earth,False,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,NaN,NaN,NaN,NaN,9269,1,1
12967,4274,9271_01,Mars,True,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,NaN,D,296.0,P,9271,1,1
12968,4275,9273_01,Europa,False,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,NaN,D,297.0,P,9273,1,1


In [168]:
categorial = ['CryoSleep', 'VIP', 'HomePlanet', 'Destination', 'Deck', 'Side']

In [169]:
ordinal = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=10000)
ordinal.fit(df[categorial])
df.loc[:,categorial] = ordinal.transform(df.loc[:,categorial])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         12970 non-null  int64  
 1   PassengerId   12970 non-null  object 
 2   HomePlanet    12682 non-null  float64
 3   CryoSleep     12660 non-null  float64
 4   Destination   12696 non-null  float64
 5   Age           12700 non-null  float64
 6   VIP           12674 non-null  float64
 7   RoomService   12707 non-null  float64
 8   FoodCourt     12681 non-null  float64
 9   ShoppingMall  12664 non-null  float64
 10  Spa           12686 non-null  float64
 11  VRDeck        12702 non-null  float64
 12  Transported   8693 non-null   object 
 13  Deck          12671 non-null  float64
 14  Num           12671 non-null  float64
 15  Side          12671 non-null  float64
 16  Group         12970 non-null  int64  
 17  Pp            12970 non-null  int64  
 18  Group_count   12970 non-nu

/tmp/ipykernel_5604/3835204275.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,categorial] = ordinal.transform(df.loc[:,categorial])


In [170]:
label = LabelEncoder()
label.fit(df.Transported)
df.loc[:,'Transported'] = label.transform(df.loc[:,'Transported'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         12970 non-null  int64  
 1   PassengerId   12970 non-null  object 
 2   HomePlanet    12682 non-null  float64
 3   CryoSleep     12660 non-null  float64
 4   Destination   12696 non-null  float64
 5   Age           12700 non-null  float64
 6   VIP           12674 non-null  float64
 7   RoomService   12707 non-null  float64
 8   FoodCourt     12681 non-null  float64
 9   ShoppingMall  12664 non-null  float64
 10  Spa           12686 non-null  float64
 11  VRDeck        12702 non-null  float64
 12  Transported   12970 non-null  int64  
 13  Deck          12671 non-null  float64
 14  Num           12671 non-null  float64
 15  Side          12671 non-null  float64
 16  Group         12970 non-null  int64  
 17  Pp            12970 non-null  int64  
 18  Group_count   12970 non-nu

/tmp/ipykernel_5604/1975810753.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,'Transported'] = label.transform(df.loc[:,'Transported'])


In [171]:
df['Age'] = df.Age.median()

In [172]:
def fill_cryo(row):
    costs = row[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum()
    if costs > 0:
        row['CryoSleep'] = 1
    else:
        row['CryoSleep'] = 0
    return row

In [173]:
df = df.apply(fill_cryo, axis=1)

In [174]:
df[df.VIP == 0].describe()

,index,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Num,Side,Group,Pp,Group_count
count,12401.000000,12122.000000,12401.000000,12137.000000,12401.0,12401.0,12149.000000,12123.000000,12107.000000,12125.000000,12142.000000,12401.000000,12120.000000,12120.000000,12120.000000,12401.000000,12401.000000,12401.000000
mean,3607.532780,0.655750,0.572938,1.495922,27.0,0.0,216.735698,419.522808,174.006608,292.849402,285.578406,1.001371,4.366832,610.400825,0.504538,4627.730425,1.512297,2.023466
std,2402.472697,0.802317,0.494671,0.812723,0.0,0.0,630.845068,1465.779835,595.481307,1090.430391,1116.953592,0.813345,1.732529,513.113530,0.500000,2682.124276,1.044894,1.584302
min,0.000000,0.000000,0.000000,0.000000,27.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
25%,1615.000000,0.000000,0.000000,1.000000,27.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,174.000000,0.000000,2306.000000,1.000000,1.000000
50%,3237.000000,0.000000,1.000000,2.000000,27.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,5.000000,445.000000,1.000000,4625.000000,1.000000,1.000000
75%,5420.000000,1.000000,1.000000,2.000000,27.0,0.0,45.000000,59.000000,26.000000,50.000000,37.000000,2.000000,6.000000,1018.000000,1.000000,6908.000000,2.000000,2.000000
max,8692.000000,2.000000,1.000000,2.000000,27.0,0.0,14327.000000,27071.000000,23492.000000,22408.000000,24133.000000,2.000000,7.000000,1894.000000,1.000000,9280.000000,8.000000,8.000000


In [175]:
df[df.VIP == 1].describe()

,index,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Num,Side,Group,Pp,Group_count
count,273.000000,267.000000,273.000000,270.000000,273.0,273.0,269.000000,267.000000,266.000000,269.000000,266.000000,273.000000,266.000000,266.000000,266.000000,273.000000,273.000000,273.000000
mean,3932.428571,1.337079,0.871795,1.288889,27.0,1.0,486.349442,1793.651685,273.680451,932.847584,1207.199248,0.820513,2.255639,281.597744,0.466165,4856.476190,1.527473,2.065934
std,2533.740898,0.473600,0.334932,0.915514,0.0,0.0,1058.272031,3593.931435,575.222273,2131.559126,2523.478754,0.831824,1.664974,372.272440,0.499794,2732.573542,0.939422,1.301601
min,2.000000,1.000000,0.000000,0.000000,27.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,1.000000,1.000000
25%,1815.000000,1.000000,1.000000,0.000000,27.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,76.000000,0.000000,2529.000000,1.000000,1.000000
50%,3599.000000,1.000000,1.000000,2.000000,27.0,1.0,1.000000,280.000000,0.000000,46.000000,28.500000,1.000000,2.000000,166.000000,0.000000,4833.000000,1.000000,2.000000
75%,6049.000000,2.000000,1.000000,2.000000,27.0,1.0,600.000000,2167.500000,277.250000,928.000000,1281.000000,2.000000,3.750000,278.750000,1.000000,7396.000000,2.000000,3.000000
max,8688.000000,2.000000,1.000000,2.000000,27.0,1.0,8243.000000,29813.000000,3700.000000,15255.000000,19086.000000,2.000000,5.000000,1791.000000,1.000000,9276.000000,6.000000,7.000000


In [176]:
df.isna().sum()

index             0
PassengerId       0
HomePlanet      288
CryoSleep         0
Destination     274
Age               0
VIP             296
RoomService     263
FoodCourt       289
ShoppingMall    306
Spa             284
VRDeck          268
Transported       0
Deck            299
Num             299
Side            299
Group             0
Pp                0
Group_count       0
dtype: int64

In [177]:
train = df[train_index]
test = df[test_index]

In [178]:
train.dropna(inplace=True)

/tmp/ipykernel_5604/167719433.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.dropna(inplace=True)


In [179]:
y_train = train.Transported
X_train = train.drop(columns=['Transported'])
X_test = test.drop(columns=['Transported'])

In [180]:
model = LGBMClassifier(random_state=RANDOM_STATE)

In [181]:
model.fit(X_train.drop(columns=['PassengerId']), y_train)

LGBMClassifier(random_state=654321)

In [182]:
pred_train = model.predict(X_train.drop(columns=['PassengerId']))

In [183]:
accuracy_score(y_train, pred_train)

0.8879334648999154

In [184]:
X_test.isna().sum()

index             0
PassengerId       0
HomePlanet       87
CryoSleep         0
Destination      92
Age               0
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Deck            100
Num             100
Side            100
Group             0
Pp                0
Group_count       0
dtype: int64

In [185]:
pred_test = model.predict(X_test.drop(columns=['PassengerId']))

In [186]:
pred_test.shape

(4277,)

In [187]:
pred = pd.concat([X_test['PassengerId'].reset_index(drop=True), pd.DataFrame(pred_test)], axis=1)
pred.columns = ['PassengerId', 'Transported']
pred

,PassengerId,Transported
0,0013_01,1
1,0018_01,0
2,0019_01,1
3,0021_01,1
4,0023_01,1
...,...,...
4272,9266_02,1
4273,9269_01,1
4274,9271_01,1
4275,9273_01,1


In [188]:
def class2bool(row):
    row['Transported'] = 'True' if row['Transported'] == 1 else 'False'
    return row

In [189]:
pred = pred.apply(class2bool, axis=1)
pred

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,True
4274,9271_01,True
4275,9273_01,True


In [194]:
pred.to_csv(f'data/attempt{str(datetime.date.today())}.csv', index=False, sep=',', header=True)